In [85]:
#from __future__ import print_function
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from scipy import ndimage
import tensorflow as tf
from PIL import Image
import requests

In [2]:
import os
# List all files in a directory using os.listdir
images=[]
equipos = ['Barcelona', 'Real Madrid', 'Valencia', 'Getafe', 'Betis', 'Atletico de Madrid', 'Athletic Bilbao']
for x in equipos:
    basepath = f'Jugadores/{x}'
    for entry in os.listdir(basepath):
        if os.path.isfile(os.path.join(basepath, entry)):
            images.append(basepath+'/'+entry)

In [6]:
tShirts = {'Camisetas':[],'Equipos':[]}
def rotateImg(x,img,num):
    tShirts['Camisetas'].append(ndimage.rotate(img, num, reshape=False))
    tShirts['Equipos'].append(x.split('/')[1])
    return tShirts

In [7]:
#tShirts = {'Camisetas':[],'Equipos':[]}
def readAndResize(images):
    for x in images:
        if '.DS_Store' not in x:
            img = cv2.imread(x)
            img = cv2.resize(img,(256,256),3)
            tShirts['Camisetas'].append(img)
            tShirts['Equipos'].append(x.split('/')[1])
            #a_flip = np.flip(img,axis=1)
            #tShirts['Camisetas'].append(a_flip)
            #tShirts['Equipos'].append(x.split('/')[1])
            #rotate270 = ndimage.rotate(img, 270, reshape=False)
            #tShirts['Camisetas'].append(rotate270)
            #tShirts['Equipos'].append(x.split('/')[1])
            #rotate90 = ndimage.rotate(img, 90, reshape=False)
            #tShirts['Camisetas'].append(rotate90)
            #tShirts['Equipos'].append(x.split('/')[1])
            #rotate45 = ndimage.rotate(img, 45, reshape=False)
            #tShirts['Camisetas'].append(rotate45)
            #tShirts['Equipos'].append(x.split('/')[1])
            rotateImg(x,img,45)
            rotateImg(x,img,90)
            rotateImg(x,img,180)
            rotateImg(x,img,270)
    return tShirts

In [5]:
readAndResize(images);

In [6]:
tShirtsDF = pd.DataFrame(tShirts)
tShirtsDF

,Camisetas,Equipos
0,"[[[16, 14, 20], [16, 14, 20], [20, 18, 24], [1...",Barcelona
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Barcelona
2,"[[[0, 0, 0], [7, 5, 5], [6, 4, 4], [5, 3, 3], ...",Barcelona
3,"[[[21, 18, 27], [18, 16, 22], [19, 17, 21], [1...",Barcelona
4,"[[[0, 0, 0], [44, 28, 16], [46, 30, 18], [43, ...",Barcelona
...,...,...
1745,"[[[43, 38, 83], [36, 30, 80], [30, 21, 76], [2...",Athletic Bilbao
1746,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Athletic Bilbao
1747,"[[[0, 0, 0], [28, 27, 23], [27, 26, 22], [25, ...",Athletic Bilbao
1748,"[[[53, 178, 128], [61, 186, 136], [61, 184, 13...",Athletic Bilbao


In [8]:
labels = {
    'Barcelona' : 0,
    'Real Madrid' : 1,
    'Valencia' : 2,
    'Getafe' : 3,
    'Betis' : 4,
    'Atletico de Madrid' : 5,
    'Athletic Bilbao' : 6
}

In [9]:
tShirtsDF['Labels'] = tShirtsDF['Equipos'].map(labels)
display(tShirtsDF)

,Camisetas,Equipos,Labels
0,"[[[16, 14, 20], [16, 14, 20], [20, 18, 24], [1...",Barcelona,0
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Barcelona,0
2,"[[[0, 0, 0], [7, 5, 5], [6, 4, 4], [5, 3, 3], ...",Barcelona,0
3,"[[[21, 18, 27], [18, 16, 22], [19, 17, 21], [1...",Barcelona,0
4,"[[[0, 0, 0], [44, 28, 16], [46, 30, 18], [43, ...",Barcelona,0
...,...,...,...
1745,"[[[43, 38, 83], [36, 30, 80], [30, 21, 76], [2...",Athletic Bilbao,6
1746,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Athletic Bilbao,6
1747,"[[[0, 0, 0], [28, 27, 23], [27, 26, 22], [25, ...",Athletic Bilbao,6
1748,"[[[53, 178, 128], [61, 186, 136], [61, 184, 13...",Athletic Bilbao,6


In [9]:
img_rows, img_cols, img_rgb = 256, 256, 3

X = np.stack(np.array(tShirtsDF.Camisetas),axis=2).swapaxes(2,0)
y = np.array(tShirtsDF.Labels)
print("Shapes X={} y={}".format(X.shape,y.shape))

classes = np.unique(y)
nClasses = len(classes)
print('Total numero de etiquetas: ', nClasses)
print('Tipos de etiquetas: ', classes)

Shapes X=(1750, 256, 256, 3) y=(1750,)
Total numero de etiquetas:  7
Tipos de etiquetas:  [0 1 2 3 4 5 6]


In [10]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)
print('Training data shape : ', X_train.shape, y_train.shape)
print('Testing data shape : ', X_test.shape, y_test.shape)
 
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

y_train = keras.utils.to_categorical(y_train, nClasses)
y_test = keras.utils.to_categorical(y_test, nClasses)

Training data shape :  (1400, 256, 256, 3) (1400,)
Testing data shape :  (350, 256, 256, 3) (350,)


In [11]:
input_shape = (img_rows, img_cols, img_rgb)
tShirts_model = Sequential()
tShirts_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
tShirts_model.add(Conv2D(64, (3, 3), activation='relu'))
tShirts_model.add(MaxPooling2D(pool_size=(2, 2)))
tShirts_model.add(Dropout(0.25))
tShirts_model.add(Flatten())
tShirts_model.add(Dense(128, activation='relu'))
tShirts_model.add(Dropout(0.5))
tShirts_model.add(Dense(nClasses, activation='softmax'))

tShirts_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [13]:
batch_size = 20
epochs = 10

tShirts_model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
score = tShirts_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Guardo los modelos que mejor me dieron

In [16]:
#tShirts_model.save("tShirts_LaLiga.h5py")  --> test accuracy de 0.75
#tShirts_model.save("tShirts073_LaLiga.h5py")  --> test accuracy de 0.73

# Hago una prueba para ver como califica a un jugador del Barcelona

In [71]:
#img2 = cv2.imread('Jugadores/Barcelona/19c9821613.jpg')
img2 = cv2.imread('./Prueba/ce8fef052b.jpg')
img2 = cv2.resize(img2,(256,256),3)
print(img2.shape)
img2 = np.expand_dims(img2, axis=0)
print(img2.shape)

(256, 256, 3)
(1, 256, 256, 3)


In [72]:
predictions_single = modelo_a_trabajar.predict(img2)
print(predictions_single)

[[0. 1. 0. 0. 0. 0. 0.]]


In [73]:
for x in predictions_single:
    if x[0] == max(x):
        print('La camiseta es del -> Barcelona')
    elif x[1] == max(x):
        print('La camiseta es del -> Real Madrid')
    elif x[2] == max(x):
        print('La camiseta es del -> Valencia')
    elif x[3] == max(x):
        print('La camiseta es del -> Getafe')
    elif x[4] == max(x):
        print('La camiseta es del -> Betis')
    elif x[5] == max(x):
        print('La camiseta es del -> Atletico de Madrid')
    elif x[6] == max(x):
        print('La camiseta es del -> Athletic Bilbao')

La camiseta es del -> Real Madrid


In [82]:
#prueba = 'Jugadores/Barcelona/19c9821613.jpg'

def reconoceLaCamiseta(path):
    def ryrImage(ruta):
        img2 = cv2.imread(ruta)
        img2 = cv2.resize(img2,(256,256),3)
        img2 = np.expand_dims(img2, axis=0)
        return img2

    resultado = ryrImage(path)

    modelo_a_trabajar = tf.keras.models.load_model(
        filepath="tShirts_LaLiga.h5py"
    )

    def predict(otraPrueba):
        predictions_single = modelo_a_trabajar.predict(otraPrueba)
        return predictions_single

    resultadoFinal = predict(resultado)
    resultadoFinal

    for x in resultadoFinal:
        if x[0] == max(x):
            print('La camiseta es del -> Barcelona')
        elif x[1] == max(x):
            print('La camiseta es del -> Real Madrid')
        elif x[2] == max(x):
            print('La camiseta es del -> Valencia')
        elif x[3] == max(x):
            print('La camiseta es del -> Getafe')
        elif x[4] == max(x):
            print('La camiseta es del -> Betis')
        elif x[5] == max(x):
            print('La camiseta es del -> Atletico de Madrid')
        elif x[6] == max(x):
            print('La camiseta es del -> Athletic Bilbao')
    return None

In [83]:
reconoceLaCamiseta('Prueba/20c6d52136.jpg')

La camiseta es del -> Barcelona


# Importo el modelo que mejor me dió

In [12]:
modelo_a_trabajar = tf.keras.models.load_model(
    filepath="tShirts_LaLiga.h5py"
)

In [54]:
equis = modelo_a_trabajar.predict(X_test)
print(equis[3])

[3.2399228e-07 2.1769578e-05 9.9983668e-01 1.6912612e-05 1.0170879e-04
 2.2311368e-05 3.2887982e-07]


In [102]:
equiposTest = pd.DataFrame(y_test)
equiposTest.columns = ['BCN', 'RMD', 'VAL', 'GET', 'BET', 'ATM', 'ATB']
equiposTest

,BCN,RMD,VAL,GET,BET,ATM,ATB
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
345,0.0,1.0,0.0,0.0,0.0,0.0,0.0
346,0.0,0.0,0.0,0.0,0.0,0.0,1.0
347,1.0,0.0,0.0,0.0,0.0,0.0,0.0
348,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [103]:
equiposTestMax = equiposTest.idxmax(axis=1)
equiposTestMax

0      VAL
1      ATM
2      GET
3      VAL
4      BCN
      ... 
345    RMD
346    ATB
347    BCN
348    RMD
349    RMD
Length: 350, dtype: object

In [104]:
prediccionesResultado = pd.DataFrame(equis)
prediccionesResultado.columns = ['BCN', 'RMD', 'VAL', 'GET', 'BET', 'ATM', 'ATB']
prediccionesResultado

,BCN,RMD,VAL,GET,BET,ATM,ATB
0,1.351515e-03,1.323944e-02,9.440190e-01,4.236745e-03,4.093752e-03,2.647957e-02,6.579968e-03
1,3.779810e-07,6.827418e-06,6.735641e-06,4.728725e-07,1.268635e-06,9.997966e-01,1.877361e-04
2,4.576966e-11,1.929950e-10,1.435207e-15,1.000000e+00,8.516760e-12,1.727124e-14,3.964205e-20
3,3.239923e-07,2.176958e-05,9.998367e-01,1.691261e-05,1.017088e-04,2.231137e-05,3.288798e-07
4,9.877312e-01,1.163369e-06,1.982204e-08,1.226405e-02,3.480409e-08,3.596043e-06,3.135274e-08
...,...,...,...,...,...,...,...
345,2.191469e-07,4.401526e-01,5.597324e-01,4.646109e-05,6.485372e-05,2.941140e-06,5.859252e-07
346,6.848549e-07,1.101461e-07,6.342930e-08,1.777337e-08,5.799563e-09,1.458493e-06,9.999976e-01
347,9.998833e-01,7.851769e-08,3.804027e-09,7.707159e-05,5.927157e-07,3.884768e-05,3.223378e-09
348,9.562062e-07,9.998028e-01,1.518510e-04,3.131110e-05,1.132607e-05,1.560756e-06,2.345764e-07


In [105]:
prediccionesResultadoMax = prediccionesResultado.idxmax(axis=1)
prediccionesResultadoMax

0      VAL
1      ATM
2      GET
3      VAL
4      BCN
      ... 
345    VAL
346    ATB
347    BCN
348    RMD
349    VAL
Length: 350, dtype: object

In [106]:
resultadoFinal = [equiposTestMax, prediccionesResultadoMax]
resultadoFinal = pd.concat(resultadoFinal, axis=1, join='inner')
resultadoFinal.columns = ['EQUIPO', 'PREDICCION']
resultadoFinal['ACIERTO'] = (resultadoFinal['EQUIPO'] == resultadoFinal['PREDICCION'])
resultadoFinal

,EQUIPO,PREDICCION,ACIERTO
0,VAL,VAL,True
1,ATM,ATM,True
2,GET,GET,True
3,VAL,VAL,True
4,BCN,BCN,True
...,...,...,...
345,RMD,VAL,False
346,ATB,ATB,True
347,BCN,BCN,True
348,RMD,RMD,True


In [107]:
resultadoFinal.ACIERTO.value_counts()

True     318
False     32
Name: ACIERTO, dtype: int64

In [108]:
porcentajeAcierto = resultadoFinal.ACIERTO.value_counts(True)
porcentajeAcierto

True     0.908571
False    0.091429
Name: ACIERTO, dtype: float64

In [90]:
x = 'Prueba/20c6d52136.jpg'
data = requests.get(f'http://localhost:5000/reconoce/{x}')
print(data)

<Response [404]>
